In [1]:
import pandas as pd
import requests
import warnings
import numpy as np
from sportsdataverse import mbb

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 120)

In [27]:
#pbp = mbb.load_mbb_pbp(range(2011,2023))

In [28]:
#pbp.groupby('participants_0_athlete_id').count().describe()

In [2]:
mbb_sc = mbb.load_mbb_schedule(range(2004,2024))
mbb_sc

100%|██████████| 20/20 [00:07<00:00,  2.63it/s]


,id,uid,date,attendance,time_valid,neutral_site,conference_competition,recent,start_date,notes_type,...,season_type,status_type_alt_detail,groups_id,groups_name,groups_short_name,groups_is_conference,tournament_id,PBP,team_box,player_box
0,244000063,s:40~l:41~e:244000063~c:244000063,2004-04-06T01:21Z,44468,True,False,False,False,2004-04-06T01:21Z,event,...,3,None,None,None,None,None,22.0,False,False,False
1,244000062,s:40~l:41~e:244000062~c:244000062,2004-04-04T01:47Z,44417,True,False,False,False,2004-04-04T01:47Z,event,...,3,None,None,None,None,None,22.0,False,False,False
2,244000031,s:40~l:41~e:244000031~c:244000031,2004-04-03T23:07Z,44417,True,False,False,False,2004-04-03T23:07Z,event,...,3,None,None,None,None,None,22.0,False,False,False
3,240920130,s:40~l:41~e:240920130~c:240920130,2004-04-02T00:00Z,16064,True,False,False,False,2004-04-02T00:00Z,event,...,3,None,None,None,None,None,NaN,False,False,False
4,240900130,s:40~l:41~e:240900130~c:240900130,2004-03-31T02:00Z,12630,True,False,False,False,2004-03-31T02:00Z,event,...,3,None,None,None,None,None,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115055,401489513,s:40~l:41~e:401489513~c:401489513,2022-11-07T21:00Z,3148,True,False,False,False,2022-11-07T21:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115056,401485025,s:40~l:41~e:401485025~c:401485025,2022-11-07T19:00Z,3380,True,False,False,False,2022-11-07T19:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115057,401494597,s:40~l:41~e:401494597~c:401494597,2022-11-07T18:00Z,175,True,False,False,False,2022-11-07T18:00Z,,...,2,None,None,None,None,None,NaN,True,True,True
115058,401483265,s:40~l:41~e:401483265~c:401483265,2022-11-07T17:00Z,10284,True,False,False,False,2022-11-07T17:00Z,,...,2,None,None,None,None,None,NaN,True,True,True


### Make boxscore trailing averages

In [3]:
mbb_bs = mbb.load_mbb_team_boxscore(range(2004,2024))

mbb_bs.team_id = mbb_bs.team_id.apply(pd.to_numeric)
mbb_bs = mbb_bs.sort_values(['team_id', 'game_date'])
mbb_bs['largest_lead'] = mbb_bs.largestLead.fillna(0).apply(lambda x: int(x)) + mbb_bs.largest_lead.fillna(0).apply(lambda x: int(x))
mbb_bs = mbb_bs.dropna(axis=1)

mbb_bs['next_gameid'] = mbb_bs.game_id.shift(-1)
mbb_bs['next_teamid'] = mbb_bs.team_id.shift(-1)
mbb_bs['next_home_away'] = mbb_bs.home_away.shift(-1)

mbb_bs[mbb_bs.team_id!=mbb_bs.next_teamid].next_gameid=-1

mbb_bs = mbb_bs.drop('next_teamid', axis=1)

mbb_bs[
        [
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ]
] = mbb_bs[
        [
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ]
    ].applymap(lambda x: x.split('-'))

mbb_bs['field_goals_made'] = mbb_bs.field_goals_made_field_goals_attempted.apply(lambda x: x[0])
mbb_bs['field_goals_attempted'] = mbb_bs.field_goals_made_field_goals_attempted.apply(lambda x: x[1])
mbb_bs['three_point_field_goals_made'] = mbb_bs.three_point_field_goals_made_three_point_field_goals_attempted.apply(lambda x: x[0])
mbb_bs['three_point_field_goals_attempted'] = mbb_bs.three_point_field_goals_made_three_point_field_goals_attempted.apply(lambda x: x[1])
mbb_bs['free_throws_made'] = mbb_bs.free_throws_made_free_throws_attempted.apply(lambda x: x[0])
mbb_bs['free_throws_attempted'] = mbb_bs.free_throws_made_free_throws_attempted.apply(lambda x: x[1])

mbb_bs = mbb_bs.drop([
            'field_goals_made_field_goals_attempted', 
            'three_point_field_goals_made_three_point_field_goals_attempted', 
            'free_throws_made_free_throws_attempted'
        ], axis=1)

mbb_bs.home_away = mbb_bs.home_away.apply(lambda x: 1 if x=='HOME' else 0)

100%|██████████| 20/20 [00:07<00:00,  2.58it/s]
/var/folders/9m/cmqzd33j5g99b6zzjsbfrvz80000gn/T/ipykernel_60717/3168056671.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbb_bs[mbb_bs.team_id!=mbb_bs.next_teamid].next_gameid=-1


In [4]:
mbb_bs['next_home_away'] = mbb_bs.home_away.shift(-1)

In [5]:
mbb_bs.groupby('team_id').count().describe()

,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,field_goal_pct,three_point_field_goal_pct,free_throw_pct,total_rebounds,...,game_date,largest_lead,next_gameid,next_home_away,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted
count,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,...,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000,1289.000000
mean,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595,...,163.897595,163.897595,163.896819,163.896819,163.897595,163.897595,163.897595,163.897595,163.897595,163.897595
std,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403,...,254.764403,254.764403,254.764901,254.764901,254.764403,254.764403,254.764403,254.764403,254.764403,254.764403
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
75%,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,...,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000,527.000000
max,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,...,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000


### Make upcoming matchups frames

In [113]:
matchups = pd.merge(mbb_bs, mbb_bs, how='inner', on='game_id', suffixes=['_act', '_opp'])
matchups = matchups[matchups.team_id_act!=matchups.team_id_opp]

ind_list = [
    'next_gameid_act',
    'next_home_away_act',

    'game_id',
    'game_date_act',
    'team_uid_act',
    'team_slug_act',
    'team_location_act',
    'team_name_act',
    'team_abbreviation_act',
    'team_display_name_act',
    'opponent_name_act',
    'opponent_mascot_act',
    'opponent_abbrev_act',
    'season_act',
    'season_type_act',
    'team_uid_opp',
    'team_slug_opp',
    'team_location_opp',
    'team_name_opp',
    'team_abbreviation_opp',
    'team_display_name_opp',
    'opponent_id_opp',
    'opponent_name_opp',
    'opponent_mascot_opp',
    'opponent_abbrev_opp',
    'season_opp',
    'season_type_opp',
    'game_date_opp',
    'next_gameid_opp',
    'next_home_away_opp',
]

upcoming = matchups.set_index(ind_list[0:2]).drop(ind_list[2:], axis=1).groupby('team_id_act').rolling(9, min_periods=9, closed='left').mean().dropna().reset_index(drop=False)
upcoming = pd.merge(upcoming[upcoming.next_home_away_act==1], upcoming[upcoming.next_home_away_act==0], how='inner', on='next_gameid_act', suffixes=['_home', '_away'])

upcoming

,team_id_act_home,next_gameid_act,next_home_away_act_home,field_goal_pct_act_home,three_point_field_goal_pct_act_home,free_throw_pct_act_home,total_rebounds_act_home,offensive_rebounds_act_home,defensive_rebounds_act_home,assists_act_home,...,flagrant_fouls_opp_away,fouls_opp_away,home_away_opp_away,largest_lead_opp_away,field_goals_made_opp_away,field_goals_attempted_opp_away,three_point_field_goals_made_opp_away,three_point_field_goals_attempted_opp_away,free_throws_made_opp_away,free_throws_attempted_opp_away
0,1,293180299.0,1.0,42.722222,34.733333,75.377778,32.666667,9.555556,20.444444,15.777778,...,0.0,17.000000,0.444444,10.666667,27.555556,56.666667,7.111111,19.888889,14.333333,20.777778
1,1,401089662.0,1.0,39.233333,32.355556,72.877778,30.666667,9.888889,20.777778,15.777778,...,0.0,16.222222,0.444444,6.777778,21.555556,52.333333,4.444444,14.222222,15.444444,23.222222
2,2,250050344.0,1.0,45.711111,33.244444,74.400000,40.444444,13.222222,23.111111,13.444444,...,0.0,18.555556,0.555556,8.111111,22.444444,54.000000,6.444444,20.444444,13.333333,18.666667
3,2,250220008.0,1.0,42.800000,28.500000,72.355556,41.111111,14.222222,22.555556,12.333333,...,0.0,17.888889,0.666667,7.444444,20.444444,49.444444,4.111111,14.666667,15.333333,22.777778
4,2,250330145.0,1.0,40.811111,30.011111,72.400000,38.444444,13.000000,21.666667,12.666667,...,0.0,16.666667,0.555556,7.333333,21.222222,45.000000,5.777778,15.444444,16.111111,23.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97799,112358,320052529.0,1.0,43.111111,29.988889,72.366667,40.777778,11.555556,29.222222,15.222222,...,0.0,17.333333,0.333333,15.222222,29.777778,64.555556,6.777778,20.666667,9.222222,15.000000
97800,112358,400841201.0,1.0,44.488889,34.411111,74.222222,40.555556,9.888889,30.111111,15.222222,...,0.0,20.000000,0.555556,8.555556,26.000000,61.333333,5.222222,19.000000,17.111111,23.777778
97801,112358,400919153.0,1.0,43.966667,35.822222,71.733333,41.444444,10.333333,30.555556,14.888889,...,0.0,16.777778,0.111111,20.333333,24.333333,56.555556,8.333333,22.888889,17.222222,24.333333
97802,112358,263322599.0,1.0,45.222222,38.255556,70.988889,40.111111,9.888889,29.666667,14.000000,...,0.0,18.222222,0.333333,8.555556,24.000000,58.444444,7.777778,23.111111,12.555556,18.888889


In [114]:
mbb_sc.id = mbb_sc.id.astype(float)
temp = upcoming.join(mbb_sc.set_index('id'), on='next_gameid_act', how='inner')
upcoming['final_spread'] = temp.home_score.astype(int) - temp.away_score.astype(int)
upcoming['home_wins'] = (upcoming.final_spread>0).astype(int)
upcoming = upcoming.dropna()

### Model training

In [115]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X = upcoming.drop(['final_spread', 'home_wins'], axis=1)
Y = upcoming.final_spread

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

clf = XGBRegressor()

clf.fit(X_train, Y_train)

mean_absolute_error(Y_test, clf.predict(X_test))

10.322383024850586

In [78]:
from sklearn.metrics import mean_squared_error
import math

mean_squared_error(Y_test, clf.predict(X_test))

169.58039154527822

In [117]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score

X = upcoming.drop(['final_spread', 'home_wins'], axis=1)
Y = upcoming.home_wins

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

clf = XGBRegressor()

clf.fit(X_train, Y_train)

precision_score(Y_test, clf.predict(X_test)>0.5)

0.7061498812471575

In [120]:
from sklearn.metrics import log_loss, accuracy_score

print(log_loss(Y_test, clf.predict(X_test)))
print(accuracy_score(Y_test, clf.predict(X_test)>0.5))

0.6296346826675115
0.6753656438199025


### Bracket for 2023 MM

In [104]:
mbb_sc[mbb_sc.home_name=='Volunteers'].sort_values('date').home_id

4938      2633
4598      2633
4383      2633
3809      2633
3715      2633
          ... 
110184    2633
110042    2633
109484    2633
109363    2633
109012    2633
Name: home_id, Length: 399, dtype: object

In [105]:
upcoming['effective_date'] = temp.date
upcoming.sort_values(by='effective_date').groupby('team_id_act_home').last()[['next_gameid_act', 'effective_date']].loc[2633]

next_gameid_act          401519353.0
effective_date     2023-03-10T20:30Z
Name: 2633, dtype: object

In [107]:
mbb_sc[mbb_sc.id==401519353].T

,108967
id,401519353.0
uid,s:40~l:41~e:401519353~c:401519353
date,2023-03-10T20:30Z
attendance,16107
time_valid,True
neutral_site,True
conference_competition,True
recent,False
start_date,2023-03-10T20:30Z
notes_type,event
